In [96]:
import wikipedia
wikipedia.set_rate_limiting(True)

In [97]:
main_page = wikipedia.page("Category:Lists of anime episodes")

In [98]:
main_page.title

'Category:Lists of anime episodes'

In [99]:
main_page.pageid

'20148056'

In [100]:
html = main_page.html()

In [101]:
from bs4 import BeautifulSoup, SoupStrainer

# http = httplib2.Http()
# status, response = http.request('http://www.nytimes.com')
links = list()

for link in BeautifulSoup(html, parse_only=SoupStrainer('a')):
    if link.has_attr('href'):
        links.append(link['href'])

In [102]:
links = links[1:]

In [103]:
anime_links

['/wiki/Help:Category',
 '#mw-head',
 '#searchInput',
 'https://en.wikipedia.org/wiki/Category:Lists_of_anime_episodes',
 'https://en.wikipedia.org/w/index.php?title=Category:Lists_of_anime_episodes&from=0',
 'https://en.wikipedia.org/w/index.php?title=Category:Lists_of_anime_episodes&from=A',
 'https://en.wikipedia.org/w/index.php?title=Category:Lists_of_anime_episodes&from=B',
 'https://en.wikipedia.org/w/index.php?title=Category:Lists_of_anime_episodes&from=C',
 'https://en.wikipedia.org/w/index.php?title=Category:Lists_of_anime_episodes&from=D',
 'https://en.wikipedia.org/w/index.php?title=Category:Lists_of_anime_episodes&from=E',
 'https://en.wikipedia.org/w/index.php?title=Category:Lists_of_anime_episodes&from=F',
 'https://en.wikipedia.org/w/index.php?title=Category:Lists_of_anime_episodes&from=G',
 'https://en.wikipedia.org/w/index.php?title=Category:Lists_of_anime_episodes&from=H',
 'https://en.wikipedia.org/w/index.php?title=Category:Lists_of_anime_episodes&from=I',
 'https:/

In [104]:
import httplib2
http = httplib2.Http()
anime_links = list()
for link in links:
    status, response = http.request(link)
    # print(response)
    for thing in BeautifulSoup(response, parse_only=SoupStrainer('a')):
        try:
            if thing.has_attr('href'):
                anime_links.append(thing['href'])
        except:
            print("failed for %s" % link)

failed for https://en.wikipedia.org/w/index.php?title=Category:Lists_of_anime_episodes&from=0
failed for https://en.wikipedia.org/w/index.php?title=Category:Lists_of_anime_episodes&from=A
failed for https://en.wikipedia.org/w/index.php?title=Category:Lists_of_anime_episodes&from=B
failed for https://en.wikipedia.org/w/index.php?title=Category:Lists_of_anime_episodes&from=C
failed for https://en.wikipedia.org/w/index.php?title=Category:Lists_of_anime_episodes&from=D
failed for https://en.wikipedia.org/w/index.php?title=Category:Lists_of_anime_episodes&from=E
failed for https://en.wikipedia.org/w/index.php?title=Category:Lists_of_anime_episodes&from=F
failed for https://en.wikipedia.org/w/index.php?title=Category:Lists_of_anime_episodes&from=G
failed for https://en.wikipedia.org/w/index.php?title=Category:Lists_of_anime_episodes&from=H
failed for https://en.wikipedia.org/w/index.php?title=Category:Lists_of_anime_episodes&from=I
failed for https://en.wikipedia.org/w/index.php?title=Catego

In [107]:
only_list_of_list = list(filter(lambda x : x.startswith('/wiki/List_of_'), anime_links))

In [110]:
len(only_list_of_list)

4372

In [116]:
only_list_of_list[5]

'/wiki/List_of_Ace_of_Diamond_episodes'

In [126]:
http = httplib2.Http()
idk_yet = list()
for link in only_list_of_list[:1]:
    print('https://en.wikipedia.org' + link)
    status, response = http.request('https://en.wikipedia.org' + link)
    # print(response)
    for thing in BeautifulSoup(response, parse_only=SoupStrainer('table')):
        print(thing)

https://en.wikipedia.org/wiki/List_of_07-Ghost_episodes
html
<table class="wikitable" style="width:98%; margin:auto; background:#FFF;">
<tbody><tr style="border-bottom: 3px solid #CCF;">
<th style="width:3em;">No.
</th>
<th>Title
</th>
<th style="width:12em;">Original air date<sup class="reference" id="cite_ref-MAD_1-1"><a href="#cite_note-MAD-1">[1]</a></sup>
</th></tr>
<tr class="vevent" style="text-align:center;background:#F2F2F2"><th id="ep1" rowspan="1" scope="row" style="text-align:center">1</th><td class="summary" style="text-align:left">"The Future of Painful Thoughts is"<br/>Transcription: "<span lang="ja-Latn" style="font-style:normal" title="Japanese-language romanization">Setsu naru Omoi no Yukusue wa</span>" (<a href="/wiki/Japanese_language" title="Japanese language">Japanese</a>: <span lang="ja">切なる想いの行く末は</span>)</td><td style="text-align:center">April 7, 2009<span style="display:none"> (<span class="bday dtstart published updated">2009-04-07</span>)</span></td></tr><tr